# BeeAI ReAct agents

The BeeAI ReAct agent is a pre-configured implementation of the ReAct (Reasoning and Acting) pattern. It can be customized with tools and instructions to suit different tasks.

The ReAct pattern is a framework used in AI models, particularly language models, to separate the reasoning process from the action-taking process. This pattern enhances the model's ability to handle complex queries by enabling it to:

- Reason about the problem.
- Decide on an action to take.
- Observe the result of that action to inform further reasoning and actions.

The ReAct agent provides a convenient out-of-the-box agent implementation that makes it easier to build agents using this pattern.

## Basic ReAct Agent

To configure a ReAct agent, you need to define a ChatModel and construct a BeeAgent.

In this example, we won't provide any external tools to the agent. It will rely solely on its own memory to provide answers. This is a basic setup where the agent tries to reason and act based on the context it has built internally.

Try modifying the input text in the call to agent.run() to experiment with obtaining different answers. This will help you see how the agent's reasoning and response vary with different prompts.

In [12]:
from beeai_framework.agents.bee.agent import BeeAgent
from beeai_framework.agents.types import BeeInput, BeeRunInput, BeeRunOutput
from beeai_framework.backend.chat import ChatModel
from beeai_framework.memory.unconstrained_memory import UnconstrainedMemory

# Construct ChatModel
chat_model: ChatModel = ChatModel.from_name("ollama:granite3.1-dense:8b")

# Construct Agent instance with the chat model
agent = BeeAgent(bee_input=BeeInput(llm=chat_model, tools=[], memory=UnconstrainedMemory()))

# Run the agent
result: BeeRunOutput = await agent.run(run_input=BeeRunInput(prompt="What chemical elements make up a water molecule?"))

print(result.result.text)

A water molecule consists of two hydrogen atoms and one oxygen atom. This can be represented as H2O.


## Using Tools

To go beyond just chatting with an LLM, you can provide tools to the agent. This enables the agent to perform specific tasks and interact with external systems, enhancing its functionality. There are different ways to add tools to the agent:

- Built-in tools from the framework: BeeAI provides several built-in tools that you can easily integrate with your agent.
- Importing tools from other libraries: You can bring in external tools or APIs to extend your agent's capabilities.
- Custom tooling: You can also write your own custom tools tailored to your specific use case.
- 
By equipping the agent with these tools, you allow it to perform more complex actions, such as querying databases, interacting with APIs, or manipulating data.

## Built-in tools

BeeAI comes with several built-in tools that are part of the library, which can be easily imported and added to your agent.

In this example, we provide the agent with a weather forecast lookup tool called OpenMeteoTool. With this tool, the agent can retrieve real-time weather data, enabling it to answer weather-related queries with more accuracy.

In [4]:
from beeai_framework.backend.chat import ChatModel
from beeai_framework.tools.weather.openmeteo import OpenMeteoTool

chat_model: ChatModel = ChatModel.from_name("ollama:granite3.1-dense:8b")

# create an agent using the default LLM and add the OpenMeteoTool that is capable of fetching weather-based information
agent = BeeAgent(bee_input=BeeInput(llm=chat_model, tools=[OpenMeteoTool()], memory=UnconstrainedMemory()))

result: BeeRunOutput = await agent.run(run_input=BeeRunInput(prompt="What's the current weather in London?"))

print(result.result.text)

The current temperature in London is 8.2 degrees Celsius with no rainfall, relative humidity of 85%, and a wind speed of 7.6 km/h.


### Custom Tools

You can also write your own custom tools from scratch and integrate them into your agent.

To define a custom tool, simply use the @tool decorator on your function. Then, pass the tool into your agent just like you would with built-in tools.

In [ ]:
import json
from urllib.parse import quote

import requests

from beeai_framework.agents.bee.agent import BeeAgent
from beeai_framework.backend.chat import ChatModel
from beeai_framework.tools import tool
from beeai_framework.tools.tool import StringToolOutput
from beeai_framework.tools.weather.openmeteo import OpenMeteoTool


# defining a tool using the `tool` decorator
# Note: the pydoc is important as it serves as the tool description to the agent
@tool
def basic_calculator(expression: str) -> str:
    """
    A calculator tool that performs mathematical operations.

    Args:
        expression: The mathematical expression to evaluate (e.g., "2 + 3 * 4").

    Returns:
        The result of the mathematical expression
    """
    try:
        encoded_expression = quote(expression)
        math_url = f"https://newton.vercel.app/api/v2/simplify/{encoded_expression}"

        response = requests.get(
            math_url,
            headers={"Accept": "application/json"},
        )
        response.raise_for_status()
        return StringToolOutput(json.dumps(response.json()))

    except Exception as e:
        raise RuntimeError(f"Error evaluating expression: {e!s}") from Exception


chat_model: ChatModel = ChatModel.from_name("ollama:granite3.1-dense:8b")
agent = BeeAgent(bee_input=BeeInput(llm=chat_model, tools=[basic_calculator], memory=UnconstrainedMemory()))
result: BeeRunOutput = await agent.run(run_input=BeeRunInput(prompt="What is the square root of 36?"))
print(result.result.text)

The square root of 36 is 6.


### Imported Tools

Tools can also be imported from other libraries to extend the functionality of your agent. For example, you can integrate tools from libraries like LangChain to give your agent access to even more capabilities.

Here’s an example showing how to integrate a tool from LangChain, written in long form (without using the @tool decorator):

In [ ]:
from typing import Any

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from pydantic import BaseModel, Field

from beeai_framework.agents.bee.agent import BeeAgent
from beeai_framework.tools import Tool


class LangChainWikipediaToolInput(BaseModel):
    query: str = Field(description="The topic or question to search for on Wikipedia.")


class LangChainWikipediaTool(Tool):
    """Adapter class to integrate LangChain's Wikipedia tool with our framework"""

    name = "Wikipedia"
    description = "Search factual and historical information from Wikipedia about given topics."
    input_schema = LangChainWikipediaToolInput

    def __init__(self) -> None:
        super().__init__()
        wikipedia = WikipediaAPIWrapper()
        self.wikipedia = WikipediaQueryRun(api_wrapper=wikipedia)

    def _run(self, input: LangChainWikipediaToolInput, _: Any | None = None) -> None:
        query = input.query
        try:
            result = self.wikipedia.run(query)
            return StringToolOutput(result=result)
        except Exception as e:
            print(f"Wikipedia search error: {e!s}")
            return f"Error searching Wikipedia: {e!s}"


chat_model: ChatModel = ChatModel.from_name("ollama:granite3.1-dense:8b")
agent = BeeAgent(bee_input=BeeInput(llm=chat_model, tools=[LangChainWikipediaTool()], memory=UnconstrainedMemory()))
result: BeeRunOutput = await agent.run(
    run_input=BeeRunInput(prompt="Who is the current president of the European Commission?")
)
print(result.result.text)

The current president of the European Commission is Ursula von der Leyen.


The previous example can be re-written in a shorter form by adding the @tool decorator, which simplifies the tool definition. Here's how you can do it:

In [ ]:
from langchain_community.tools import WikipediaQueryRun  # noqa: F811
from langchain_community.utilities import WikipediaAPIWrapper  # noqa: F811

from beeai_framework.agents.bee.agent import BeeAgent
from beeai_framework.tools import Tool


# defining a tool using the `tool` decorator
# Note: the pydoc is important as it serves as the tool description to the agent
@tool
def langchain_wikipedia_tool(expression: str) -> str:
    """
    Search factual and historical information, including biography, history, politics, geography, society, culture,
    science, technology, people, animal species, mathematics, and other subjects.

    Args:
        expression: The topic or question to search for on Wikipedia.

    Returns:
        The information found via searching Wikipedia.
    """
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    return StringToolOutput(wikipedia.run(expression))


# using the tool in an agent
chat_model: ChatModel = ChatModel.from_name("ollama:granite3.1-dense:8b")
agent = BeeAgent(bee_input=BeeInput(llm=chat_model, tools=[langchain_wikipedia_tool], memory=UnconstrainedMemory()))
result: BeeRunOutput = await agent.run(
    run_input=BeeRunInput(prompt="Who is the current president of the European Commission?")
)
print(result.result.text)

The current president of the European Commission is Ursula von der Leyen.
